In [17]:
from env import Env
import openai
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials 


In [3]:
# load API keys from .env file
openai.api_key = Env.OPEN_AI_KEY
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

## Create playlist using ChatGPT
Create seed playlist with ChatGPT for song characteristic analysis.

**\#PromptEngineering \#ChatGPT**

In [4]:
response=openai.ChatCompletion.create(
    model='gpt-4',
    messages=[
        {
            'role':'system',
            'content':
            """
            You are a music playlist designer.  You create playlists for people based on different inputs.

            Output all playlists in the following JSON format: 

            {
                "playlist_name": "<name>",
                "playlist_description": "<description>",
                "playlist_tracks": [
                    "<track_name_1>",
                    "<track_name_2>",
                    ...
                ]
            }

            Replace <track_name_n> with just the name of the song (do not add the artist) that you want in the playlist. 

            If you don't know what to do, type "help". 

            I want you to return in a json format only and not any extra text. 
            

            """
        },
        {
            'role':'user',
            'content':'Can you create me a high tempo playlist of 30 songs?'
        }

    

    ]

)
print(response.choices[0]['message']['content'])

{
  "playlist_name": "High Tempo Hits",
  "playlist_description": "A collection of 30 high tempo songs guaranteed to keep your energy skyrocketing.",
  "playlist_tracks": [
    "Sandstorm",
    "Levels",
    "Turn Down for What",
    "Bangarang",
    "I Love It",
    "Turbulence",
    "Pump It",
    "Party Rock Anthem",
    "Wake Me Up",
    "Runaway (U & I)",
    "Calling (Lose My Mind)",
    "Titanium",
    "Don't You Worry Child",
    "Hey Boy, Hey Girl",
    "Animals",
    "Fade Into Darkness",
    "Show Me Love",
    "Mammoth",
    "This Is What It Feels Like",
    "One (Your Name)",
    "Internet Friends",
    "Rolling In The Deep",
    "Nothing But The Beat",
    "Find You",
    "Lovers On The Sun",
    "More Than You Know",
    "Hyperreal",
    "Wake Up Call",
    "Rock & Roll",
    "Pressure"
  ]
}


## Enrich songs with Spotify data
Use the Spotify API to add features to the songs for further analysis.

**\#WebAPI \#DataEnrichment**

In [19]:
# extract songs from text response
songs=json.loads(response.choices[0]['message']['content'])['playlist_tracks']

# filter song attributes from spotify
filter_song = lambda song: ({
    'id': song['id'],
    'name': song['name'],
    'artist': song['artists'][0]['name'],
    'album': song['album']['name'],
    'length': song['duration_ms'],
    'explicit': song['explicit'],
    'popularity': song['popularity'],
    'type': song['type']
}) 

# get the spotify song ids
spotify_songs = [ 
    filter_song(spotify.search(song)['tracks']['items'][0]) 
    for song in songs 
]

# get the spotify song features
spotify_songs = [
    {**song, **spotify.audio_features(song['id'])[0]}
    for song in spotify_songs
]

spotify_songs_df = pd.DataFrame(spotify_songs)
spotify_songs_df 

,id,name,artist,album,length,explicit,popularity,type,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,uri,track_href,analysis_url,duration_ms,time_signature
0,6Sy9BUbgFse0n0LPA5lwy5,Sandstorm,Darude,"Before the Storm, Special Edition",225493,False,72,audio_features,0.528,0.965,...,0.141000,0.985000,0.0797,0.5870,136.065,spotify:track:6Sy9BUbgFse0n0LPA5lwy5,https://api.spotify.com/v1/tracks/6Sy9BUbgFse0...,https://api.spotify.com/v1/audio-analysis/6Sy9...,225493,4
1,6hgTMdO3CyXlNQJ1K58ruH,Levels,BigXthaPlug,AMAR,157377,True,73,audio_features,0.801,0.555,...,0.114000,0.000000,0.5610,0.4900,121.935,spotify:track:6hgTMdO3CyXlNQJ1K58ruH,https://api.spotify.com/v1/tracks/6hgTMdO3CyXl...,https://api.spotify.com/v1/audio-analysis/6hgT...,157377,4
2,67awxiNHNyjMXhVgsHuIrs,Turn Down for What,DJ Snake,Turn Down for What,213733,False,72,audio_features,0.818,0.799,...,0.001070,0.128000,0.0570,0.0815,100.014,spotify:track:67awxiNHNyjMXhVgsHuIrs,https://api.spotify.com/v1/tracks/67awxiNHNyjM...,https://api.spotify.com/v1/audio-analysis/67aw...,213733,4
3,6VRhkROS2SZHGlp0pxndbJ,Bangarang (feat. Sirah),Skrillex,Bangarang EP,215253,True,71,audio_features,0.716,0.972,...,0.014500,0.000032,0.3170,0.5760,110.026,spotify:track:6VRhkROS2SZHGlp0pxndbJ,https://api.spotify.com/v1/tracks/6VRhkROS2SZH...,https://api.spotify.com/v1/audio-analysis/6VRh...,215253,4
4,6HZ9VeI5IRFCNQLXhpF4bq,I Love It (feat. Charli XCX),Icona Pop,THIS IS... ICONA POP,157152,True,72,audio_features,0.711,0.906,...,0.009520,0.000016,0.1530,0.8240,125.916,spotify:track:6HZ9VeI5IRFCNQLXhpF4bq,https://api.spotify.com/v1/tracks/6HZ9VeI5IRFC...,https://api.spotify.com/v1/audio-analysis/6HZ9...,157153,4
5,1dJzTNm46rtvQR2A3GZU8F,Turbulence,P!nk,TRUSTFALL,206453,False,64,audio_features,0.530,0.607,...,0.227000,0.000000,0.0970,0.1780,123.891,spotify:track:1dJzTNm46rtvQR2A3GZU8F,https://api.spotify.com/v1/tracks/1dJzTNm46rtv...,https://api.spotify.com/v1/audio-analysis/1dJz...,206453,4
6,6btyEL6NwUa97Nex9cZFvo,Pump It,Black Eyed Peas,Monkey Business,213066,True,70,audio_features,0.649,0.931,...,0.009300,0.000000,0.7510,0.7440,153.645,spotify:track:6btyEL6NwUa97Nex9cZFvo,https://api.spotify.com/v1/tracks/6btyEL6NwUa9...,https://api.spotify.com/v1/audio-analysis/6bty...,213067,4
7,0IkKz2J93C94Ei4BvDop7P,Party Rock Anthem,LMFAO,Sorry For Party Rocking,262173,False,71,audio_features,0.750,0.727,...,0.018900,0.000000,0.2660,0.3590,129.993,spotify:track:0IkKz2J93C94Ei4BvDop7P,https://api.spotify.com/v1/tracks/0IkKz2J93C94...,https://api.spotify.com/v1/audio-analysis/0IkK...,262173,4
8,0nrRP2bk19rLc0orkWPQk2,Wake Me Up,Avicii,True,247426,False,87,audio_features,0.532,0.783,...,0.003800,0.001200,0.1610,0.6430,124.080,spotify:track:0nrRP2bk19rLc0orkWPQk2,https://api.spotify.com/v1/tracks/0nrRP2bk19rL...,https://api.spotify.com/v1/audio-analysis/0nrR...,247427,4
9,46lFttIf5hnUZMGvjK0Wxo,Runaway (U & I),Galantis,Pharmacy,227073,False,77,audio_features,0.506,0.805,...,0.007110,0.001930,0.0856,0.3830,126.008,spotify:track:46lFttIf5hnUZMGvjK0Wxo,https://api.spotify.com/v1/tracks/46lFttIf5hnU...,https://api.spotify.com/v1/audio-analysis/46lF...,227074,4
